In [1]:
import requests
import csv
import json
import re
import ast

In [4]:
url = 'https://nookplaza.net/main.dart.js?v=15'
r = requests.get(url)
mainJs = r.text

In [94]:
# 'C.Iz=new H.b(13,{},C.a,[P.a,P.a])'
m = re.findall('new H\.b\(13,(.*),C\.a,\[P\.a,P\.a\]\)', mainJs)
print(len(m))

m = [item.encode("utf-8").decode('unicode-escape') for item in m]
m = [ast.literal_eval(item) for item in m if not 'null' in item]
print(len(m))

nameMap = {}
for item in m:
    nameMap[item["en-us"].lower()] = item

5715
5713


In [113]:
with open('../tempData/en2tw.txt', 'rt', encoding='utf-8', newline='') as fin:
    cin = csv.reader(fin, delimiter='	')
    en2twList = [row for row in cin]
print(len(en2twList))

en2tw = {}
for en, tw in en2twList:
    en2tw[en.lower()] = tw
    
def en2twFn(en):
    if en.lower() in en2tw:
        return en2tw[en.lower()]
    elif en.lower() in nameMap:
        return nameMap[en.lower()]['zh-tw']
    return en

808


In [110]:
en2tw['housewares']

'家具'

In [24]:
url = 'https://api.nookplaza.net/items?category=Furniture'
r = requests.get(url)
data = r.json()['results']

In [53]:
url = 'https://api.nookplaza.net/items?category=Recipes'
r = requests.get(url)
DIYs = r.json()['results']
DIYMap = {}

for d in DIYs:
    DIYMap[d['name'].lower()] = d

In [101]:
en2twFn('housewares')

'housewares'

In [114]:
url = 'https://api.nookplaza.net/items?category=Furniture'
r = requests.get(url)
data = r.json()['results']

idMap = {}
dictName = ['id', 'name_e', 'name_j', 'name_c', 'buy', 'sell', 'category', 'tag', 'themes', 'interact', 'size', 'obtainedFrom', 'DIY', 'diyInfo']
dictName += ['filename', 'bodyTitle', 'patternTitle', 'bodyCustomize', 'patternCustomize', 'bodys', 'pattrens', 'variations']

output = [['id', '名稱(英文)', '名稱(日文)', '名稱(中文)', '買價', '賣價', '分類', '標籤', '主題', '可互動', '大小', '取得方式', 'DIY', 'DIY資訊', '圖片', '款式說明', '樣式說明', '款式改造', '樣式改造', '款式', '樣式', '檔名對照索引']]
for item in data:
#     if item['name'].lower() == 'saw stag model':
#         print(item)
    if item["category"] == "Art":
        continue
    name = nameMap[item['name'].lower()]
    nameMap[item['name'].lower()]['id'] = item['content']['internalID']
    idMap[item['content']['internalID']] = item

#     print(item['content'])
    item['content']['category_'] = en2twFn(item['content']['category'])
    item['content']['tag_'] = en2twFn(item['content']['tag'])
    item['content']['themes_'] = [en2twFn(theme) for theme in item['content']['themes']]
    item['content']['obtainedFrom_'] = [en2twFn(item) for item in item['content']['obtainedFrom']]
    
    # 名稱 買價 賣價
    row = [int(item['content']['internalID']), name['en-us'], name['ja-jp'], name['zh-tw'], item['content']['buy'], item['content']['sell']]

    # 分類 標籤 主題
    # 'category' 'tag' 'themes'
    if "無" in item['content']['themes_']:
        item['content']['themes_'].remove("無")

    row += [item['content']['category_'], item['content']['tag_'], '無' if len(item['content']['themes_']) == 0 else '、'.join(item['content']['themes_'])]
    
    # 互動 大小
    row += [item['content']['interact'], '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])]

    # 取得方式 DIY
    if not 'dIY' in item['content']:
        print(item)
    row += ['、'.join(item['content']['obtainedFrom_']), item['content']['dIY']]
    
    diyInfo = {}
    if item['content']['dIY']:
        diyInfo_tem = DIYMap[item['name'].lower()]['content']
        diyInfo['materials'] = [{'count': item['count'], 'itemName': en2twFn(item['itemName'].lower())} for item in diyInfo_tem['materials']]
        diyInfo['obtainedFrom'] = '、'.join(diyInfo_tem['obtainedFrom'])
        diyInfo['sourceNotes'] = diyInfo_tem['sourceNotes']
    
    row += [diyInfo]
    
    # 圖片檔名
    row += [item['content']['filename']]
    
    # 改造-款式 樣式
    # bodyCustomize bodyColor bodyTitle; patternCustomize patternColor patternTitle
    row += [item['content']['bodyTitle'], item['content']['patternTitle'], item['content']['bodyCustomize'], item['content']['patternCustomize']]

    variations = {
        "bodys": {},
        "pattrens": {}
    }
    for variation in item['variations']:
        content = variation['content']
        filename = content['filename']
        bodyIndex, pattrenIndex = filename.split('_')[-2:]
        
        bodyColor = '' if content['bodyColor'] is None else content['bodyColor']
        patternColor = '' if content['patternColor'] is None else content['patternColor']
        
        variations['bodys'][bodyColor] = bodyIndex
        variations['pattrens'][patternColor] = pattrenIndex

#         response = requests.get(content['image'])
#         with open('../img/furniture/{}.png'.format(filename), "wb") as fout:
#             fout.write(response.content)
        
    row += ['、'.join(list(variations['bodys'].keys())), '、'.join(list(variations['pattrens'].keys())), variations]
       
    if item['name'].lower() == 'classic pitcher':
        print(row)
    output.append(row)
output[1:] = sorted(output[1:], key=lambda k: k[0])

[4073, 'classic pitcher', 'とうきのジャグ', '陶製牛奶壺', None, 800, '小物件', '餐飲', '廚房、商店', False, '1.0x1.0', 'DIY', True, {'materials': [{'count': 4, 'itemName': '黏土'}], 'obtainedFrom': 'Tom Nook、Snooty villagers', 'sourceNotes': 'Obtained during the main storyline during the villager house development quest'}, 'FtrPotteryjug_Remake_0_0', 'Design', None, True, False, 'Unglazed、Simple、Sky blue、Yellow amber、Fruits、Artistic', '', {'bodys': {'Unglazed': '0', 'Simple': '1', 'Sky blue': '2', 'Yellow amber': '3', 'Fruits': '4', 'Artistic': '5'}, 'pattrens': {'': '0'}}]


In [115]:
with open('items.csv', 'wt', encoding='utf-8', newline='') as fout:
    csvout = csv.writer(fout)
    csvout.writerows(output)

jsonOutput = []
for row in output[1:]:
    j = {}
    for index, item in enumerate(row):
        j[dictName[index]] = item
    jsonOutput.append(j)

with open('../../nook_link/src/data/furniture.json', 'wt') as fout:
    fout.write(json.dumps(jsonOutput))
    
with open('furniture.json', 'wt') as fout:
    fout.write(json.dumps(jsonOutput))